### Άνοιγμα αρχείου και προεργασία

In [ ]:
import numpy as np
import csv
from IPython.display import display, Markdown
from sklearn.preprocessing import StandardScaler


results = []
with open("abalone.data") as myfile:
  data = csv.reader(myfile, delimiter=',')
  for dataline in data:
    results.append(dataline)



for element in results:
  element.pop(0)
y = []
for element in results:
  target = element.pop(7)
  y.append(target)

XArray = np.array(results)
yArray = np.array(y)

XArray = XArray.astype(float)
yArray = yArray.astype(int)

# print(15*"==========")
# print(XArray)
# print(15*"==========")
# print(yArray)

display(Markdown(
    "**Number of objects**: " + str(len(XArray)) + "<br>" +
    "**Number of variables**: " + str(len(XArray[0]))  + "<br>" +
    "**Range of y**: " + str(np.min(yArray)) + "-" + str(np.max(yArray)) + "<br>" +
    "**Test data mean**: " + str(np.mean(XArray,0)) + "<br>" +
    "**Test data std**: " + str(np.std(XArray,0))
))


from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(XArray,yArray,test_size = 0.3, random_state=5)

print(np.shape(X_train), np.shape(X_test))


# normalize 
datascaler = StandardScaler().fit(X_train)
X_train_normalized = datascaler.transform(X_train)
X_test_normalized = datascaler.transform(X_test)

display(Markdown(
    "**Normalized Training data mean**: " + str(np.mean(X_train_normalized,0)) + "<br>" +
    "**Normalized Training data std**: " + str(np.std(X_train_normalized,0))  + "<br>" +
    "**Normalized Test data mean**: " + str(np.mean(X_test_normalized,0)) + "<br>" +
    "**Normalized Test data std**: " + str(np.std(X_test_normalized,0))
))


**Number of objects**: 4177<br>**Number of variables**: 7<br>**Range of y**: 1-29<br>**Test data mean**: [0.5239921  0.40788125 0.1395164  0.82874216 0.35936749 0.18059361
 0.23883086]<br>**Test data std**: [0.12007854 0.09922799 0.04182205 0.49033031 0.22193638 0.10960113
 0.13918601]

(2923, 7) (1254, 7)


**Normalized Training data mean**: [-4.90176949e-15 -8.34942409e-15  1.44026084e-15 -1.66273275e-15
  6.54492236e-16 -2.23059682e-15  6.34598999e-16]<br>**Normalized Training data std**: [1. 1. 1. 1. 1. 1. 1.]<br>**Normalized Test data mean**: [ 0.00254309  0.01051423 -0.00311952  0.00492772 -0.00922393  0.013566
  0.0033401 ]<br>**Normalized Test data std**: [0.98836835 0.99495613 0.88021946 0.9881448  0.97133766 1.00922178
 1.002911  ]

### Κριτήριο αξιολόγησης αποτελεσμάτων

In [ ]:
#Μ.Ζ.: Αντί για δύο διαδοχικά loops, θα μπορούσατε να κάνετε np.count_nonzeros(np.less_equal(np.abs(pred-y),N)). Στην python το loop επί των δεδομένων αντενδείκνυται. Δεν αφαιρέθηκαν βαθμοί.
def success_rate(y_test_pred,y_test,N):
  

  pred_minus_test = []
  for i in range (len(y_test)):
    pred_minus_test.append(np.absolute(y_test_pred[i] - y_test[i]))


  successes = 0
  for value in np.less_equal(pred_minus_test,N):
    if value == True:
      successes += 1

  success = (successes * 100) / len(y_test_pred)

  return success

### Γραμμικά μοντέλα

In [ ]:
from sklearn import linear_model
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error, r2_score

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

display(Markdown(
    "**Mean squared error**: " + str(mean_squared_error(y_test, y_pred))  + "<br>" +
    "**Explained Variance**: " + str(explained_variance_score(y_test, y_pred)) + "<br>" +
    "**Success_rate**: " + str(success_rate(y_pred, y_test, 1))
))

#Train the model using the normalized training sets
regr_normalized = linear_model.LinearRegression()

regr_normalized.fit(X_train_normalized, y_train)
y_pred_normalized = regr_normalized.predict(X_test_normalized)

display(Markdown(
    "**Explained Variance with Normalized data**: " + str(explained_variance_score(y_test, y_pred_normalized)) + "<br>" +
    "**Success_rate with Normalized data**: " + str(success_rate(y_pred_normalized, y_test, 1))
))

#Συμπέρασμα
print("Δεν φαίνεται να υπάρχει κάποια σημαντική διαφορά ανάμεσα στα αποτελέσματα που παίρνουμε εκπαιδεύοντας με τα κανονικοποιημένα δεδομένα. ")



**Mean squared error**: 5.315759509826538<br>**Explained Variance**: 0.4849327826979455<br>**Success_rate**: 40.35087719298246

**Explained Variance with Normalized data**: 0.4849327826979454<br>**Success_rate with Normalized data**: 40.35087719298246

Δεν φαίνεται να υπάρχει κάποια σημαντική διαφορά ανάμεσα στα αποτελέσματα που παίρνουμε εκπαιδεύοντας με τα κανονικοποιημένα δεδομένα. 


In [ ]:
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score

# Ridge regression model with grid search
params = { 
           'alpha': np.logspace(-6, 4, 20) # try 20 values in logarithmic scale from 10^-6 to 10^4
         }

ridge_regr = linear_model.Ridge()
grid = GridSearchCV(estimator=ridge_regr,cv=10,param_grid=params)
grid.fit(X_train, y_train) 
y_train_pred = grid.predict(X_train)
y_test_pred = grid.predict(X_test)

display(Markdown(
    "**Ridge Regression**: <br>" +
    "**Best lambda**: {:.3f}".format(grid.best_params_['alpha'])  + "<br>" +
    "**Cross-val explained variance**: {:.2f}".format(explained_variance_score(y_train, y_train_pred)) + "<br>" +
    "**Test set explained variance**: {:.2f}".format(explained_variance_score(y_test, y_test_pred)) + "<br>" +
    "**Explained Variance**: " + str(explained_variance_score(y_test, y_test_pred)) + "<br>" +
    "**Success_rate**: " + str(success_rate(y_test_pred, y_test, 1)) + "<br>" +
    "**Coefficients from all data**: " + str(regr.coef_)
))

#Συμπέρασμα
print("Δεν φαίνεται να υπάρχει κάποια βελτίωση. Επίσης η μεταβλητή Length φαίνεται να αποκλείστηκε ως μη σημαντική ")





**Ridge Regression**: <br>**Best lambda**: 0.616<br>**Cross-val explained variance**: 0.54<br>**Test set explained variance**: 0.48<br>**Explained Variance**: 0.4812876314141211<br>**Success_rate**: 40.111642743221694<br>**Coefficients from all data**: [ -0.6067254   11.87284066   8.6632592    8.49978707 -19.14894761
 -10.11729792  11.23059286]

Δεν φαίνεται να υπάρχει κάποια βελτίωση. Επίσης η μεταβλητή Length φαίνεται να αποκλείστηκε ως μη σημαντική 


### Μη-γραμμικά μοντέλα

In [ ]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

#Create a 2nd degree polynomial model upon which to fit the data
polyModel = PolynomialFeatures(degree = 2)
X_train_poly = polyModel.fit_transform(X_train)
X_test_poly = polyModel.fit_transform(X_test)

print("Training data shape:", np.shape(X_train_poly))
print("Test data shape:", np.shape(X_test_poly))

ridge_regr_poly  = linear_model.Ridge() 
grid = GridSearchCV(estimator=ridge_regr_poly, cv = 10, param_grid=params) 
grid.fit(X_train_poly, y_train) 
y_train_pred = grid.predict(X_train_poly)
y_test_pred = grid.predict(X_test_poly)

display(Markdown(
    "**Ridge Regression**: <br>" +
    "**Best lambda**: {:.3f}".format(grid.best_params_['alpha'])  + "<br>" +
    "**Cross-val explained variance**: {:.2f}".format(explained_variance_score(y_train, y_train_pred)) + "<br>" +
    "**Test set explained variance**: {:.2f}".format(explained_variance_score(y_test, y_test_pred)) + "<br>" + 
    "**Success_rate**: " + str(success_rate(y_test_pred, y_test, 1))
))

#Σχόλια
print("Φαίνεται να υπάρχει σημαντική βελτίωση. ")


Training data shape: (2923, 36)
Test data shape: (1254, 36)


**Ridge Regression**: <br>**Best lambda**: 0.183<br>**Cross-val explained variance**: 0.58<br>**Test set explained variance**: 0.53<br>**Success_rate**: 42.66347687400319

Φαίνεται να υπάρχει σημαντική βελτίωση. 


**Ridge Regression**: <br>**Best lambda**: 23.357<br>**Cross-val explained variance**: 0.57<br>**Test set explained variance**: 0.51<br>**Success_rate**: 42.18500797448166

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
KN_regr_poly = KNeighborsRegressor()
params = { 
           'n_neighbors': range(1,10),
            'weights': ['uniform', 'distance']
         }
grid = GridSearchCV(estimator=KN_regr_poly, cv = 10, param_grid=params) 
grid.fit(X_train, y_train) 
print(grid.best_score_)
y_test_pred = grid.predict(X_test)
display(Markdown(
    "**k-Nearest Neighbours Regression**: <br>" +
    "**Best k**: {:.3f}".format(grid.best_params_['n_neighbors'])  + "<br>" +
    "**Best weighting strategy**: " + grid.best_params_['weights']  + "<br>" +
    "**Test set explained variance**: {:.2f}".format(explained_variance_score(y_test, y_test_pred)) + "<br>" + 
    "**Success_rate**: " + str(success_rate(y_test_pred, y_test, 1))
))

#Σχόλια
print("Δεν υπάρχει βελτίωση σε σχέση με την προηγούμενη μέθοδο 'Ridge Regression', αλλά υπάρχει σε σχέση με τα Γραμμικά μοντέλα.")
print("Το συμπέρασμα που βγάζουμε για τη σχέση μεταξύ των χαρακτηριστικών και του στόχου είναι ότι είναι πολυωνυμική ως 2ου βαθμού.")

0.5466731628762506


**k-Nearest Neighbours Regression**: <br>**Best k**: 9.000<br>**Best weighting strategy**: distance<br>**Test set explained variance**: 0.51<br>**Success_rate**: 46.17224880382775

Δεν υπάρχει βελτίωση σε σχέση με την προηγούμενη μέθοδο 'Ridge Regression', αλλά υπάρχει σε σχέση με τα Γραμμικά μοντέλα.
Το συμπέρασμα που βγάζουμε για τη σχέση μεταξύ των χαρακτηριστικών και του στόχου είναι ότι είναι πολυωνυμική ως 2ου βαθμού.
